## Match City to MSA (geospatially)

The goal is to create a simple "crosswalk" csv file that matches a city's geo_id to an MSA's geo_id. The file will also list the city's name and the MSA's name.

In [72]:
# Import libraries

import warnings

# Suppress unnecessary Shapely warning
warnings.filterwarnings('ignore',
                        '.*Shapely GEOS version.*')

import numpy as np
import pandas as pd
import geopandas as gp
import shapely
import pygeos
import matplotlib.pyplot as plt


# Set up Pandas defaults
pd.options.display.float_format = '{:.4f}'.format
pd.set_option("display.max_columns", None)


In [27]:
### TEMP Read in a city csv and make it a helper dataset

# Call in the population city dataset
temp_city = pd.read_csv("datasets/cleaned_census_api_files/city_data/population_city.csv",
                        dtype={'geo_id':str})

# Rename columns
temp_city = temp_city[['name','geo_id']]
temp_city.rename(columns={'geo_id':'city_geo_id','name':'city_name'}, inplace=True)

# Remove the "city/town/borough"
temp_city['city_name'] = temp_city[
    'city_name'].str.replace(
    " city| town| village| borough", "", regex=True)

# Save as a helper dataset
temp_city.to_csv("datasets/helper_datasets/city_geoid_codes.csv")

### END OF MAKING HELPER CITY GEOID DATASET

### BEGIN MAKING MSA TO CITY CROSSWALK FILE
WARNING, THIS TAKES 40 MINUTES

In [63]:
# Call in msa_geo and city_geo
msa_geo = gp.read_file("datasets/census_original_files/census_geopackages/msa_geometries.gpkg",
                       dtype={"GEOID":str})
city_geo = gp.read_file("datasets/census_original_files/census_geopackages/city_geometries.gpkg",
                       dtype={"GEOID":str})

# Rename columns
msa_geo.rename(columns={'GEOID':'msa_geoid','NAME':'msa_name'}, inplace=True)
city_geo.rename(columns={'GEOID':'city_geoid','NAME':'city_name'}, inplace=True)

# Enforce CRS
msa_geo = msa_geo.to_crs(epsg=3857)
city_geo = city_geo.to_crs(epsg=3857)

display(msa_geo.head(1))
display(city_geo.head(1))


In [87]:
# Create empty dataframe to concat
crosswalk = pd.DataFrame(
    columns=['msa_geoid','msa_name','city_geoid','city_name'],
    data=None
)

# Loop through all msa's
for i in range(len(msa_geo.index)):

    # Test 1 iteration of overlays
    test_msa = msa_geo.iloc[[i]].copy()
    test_city = city_geo.copy()

    # Create area clumn for cities
    test_city['original_area'] = test_city.area

    # Create overlay
    overlay = test_msa.overlay(test_city, 
                               how="intersection",
                               keep_geom_type=False)

    # Create new geometry
    overlay['new_geom'] = overlay.area

    # Create percent in msa
    overlay['percent_in_msa'] = 100 * overlay['new_geom']/overlay['original_area']

    # Only keep cities whose area is 50% or more within an msa boundary
    overlay = overlay[overlay['percent_in_msa']>=50].reset_index(drop=True)

    # Only keep certain cities
    overlay = overlay[['msa_geoid','msa_name','city_geoid','city_name']]

    # Add to crosswalk
    crosswalk = pd.concat([crosswalk, overlay]).reset_index(drop=True)

crosswalk



CPU times: user 23min 19s, sys: 20.1 s, total: 23min 40s
Wall time: 25min 27s


,msa_geoid,msa_name,city_geoid,city_name
0,12020,"Athens-Clarke County, GA",1380788,Watkinsville
1,12020,"Athens-Clarke County, GA",1340532,Hull
2,12020,"Athens-Clarke County, GA",1313212,Carlton
3,12020,"Athens-Clarke County, GA",1317552,Colbert
4,12020,"Athens-Clarke County, GA",1319084,Comer
...,...,...,...,...
17051,49180,"Winston-Salem, NC",3762580,Smithtown
17052,49180,"Winston-Salem, NC",3763360,Southmont
17053,49180,"Winston-Salem, NC",3769020,Tyro
17054,49180,"Winston-Salem, NC",3771760,Welcome


In [88]:
# Enforce dtypes
crosswalk['msa_geoid'] = crosswalk['msa_geoid'].astype(str)
crosswalk['city_geoid'] = crosswalk['city_geoid'].astype(str)

# Save file
crosswalk.to_csv("datasets/helper_datasets/msa_to_city_crosswalk.csv")

# END OF MAKING CROSSWALK

---

In [48]:
### BEGIN CONVERTING ALL SHP FILES TO GEOPACKAGES - TEMPORARY CODE

# msa_shape = gp.read_file("datasets/census_original_files/msa_2020/tl_2020_us_cbsa.shp",
#                          dtype={'GEOID':str, 'CSAFP':str, 'CBSAFP':str})

# # Remove all Micro Areas
# msa_shape = msa_shape[msa_shape['LSAD'] != 'M2'].reset_index(drop=True)

# # Keep certain columns
# msa_shape = msa_shape[['GEOID','NAME','geometry']]

# # Save as geopackage
# msa_shape.to_file("datasets/census_original_files/msa_2020/msa_geometries.gpkg",
#                   driver="GPKG")

# msa_shape

In [51]:
# # ### Continue converting shp files to geopackages

# shape = gp.read_file("datasets/census_original_files/msa_2020/tl_2020_us_cbsa.shp",
#                          dtype={'GEOID':str, 'STATEFP':str, 'PLACEFP':str})

# shape

